In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append("../")
sys.path.append("../src")

In [3]:
from src.common.utils import make_cm, plot_confusion_matrix
import numpy as np
import pickle

In [4]:
from src.tune.spaces import tab_spaces

scores = {
    "clf": ["precision", "recall", "f1_score", "roc_auc"],
    "reg": ["mae", "r2"],
    "rac": ["mae", "r2", "precision", "recall", "f1_score", "roc_auc"]
}

model_arch = ["knn", "mlp", "rf", "gb"]
params = {arch: list(tab_spaces[arch].keys()) for arch in model_arch}

best_score_by = {
    "clf": "roc_auc",
    "reg": "r2",
    "rac": "roc_auc"
}

In [5]:
from common.utils import reg_to_clf_target

def get_best_model(res, method):
    return max(res, key=lambda x: res[x]["scores"][best_score_by[method]][0])

def get_params_latex_table(res, method):
    best_model = get_best_model(res, method)
    for i, name in enumerate(res.keys()):
        model = name.split("-")[-5]
        if name == best_model:
            print("{\\bfseries", name[:-16], "} & ", end="")
        else:
            print(f"{name[:-16]} & ", end="")
        for param in params[model]:
            p = res[name]["params"][param]
            if isinstance(p, float):
                print(f"{p:.5f}", end="")
            else:
                print(p, end="")
            if param != params[model][-1]:
                print(" & ", end="")
        print(" \\\\")
        if i % 6 == 5:
            print("\hline")
    
def get_scores_latex_table(res, method):
    best_model = get_best_model(res, method)
    for i, name in enumerate(res.keys()):
        if name == best_model:
            print("{\\bfseries", name[:-16], "} & ", end="")
        else:
            print(f"{name[:-16]} & ", end="")
        for score in scores[method]:
            if name == best_model:
                print("{\\bfseries ", end="")
            m, s = res[name]["scores"][score]
            print(f"{m:.5f} ± {s:.5f}", end="")
            if name == best_model:
                print("}", end="")
            if score != scores[method][-1]:
                print(" & ", end="")
        print(" \\\\")
        if i % 6 == 5 and i != len(res.keys()) - 1:
            print("\hdashline")
    print("\hline")

In [7]:
with open("../results/{}.pkl", "rb") as file:
    res = pickle.load(file)

get_scores_latex_table(res, "clf")

# res[get_best_model(res, "clf")]["params"]

{\bfseries grey-mlp-clf } & {\bfseries 0.31250 ± 0.07122} & {\bfseries 0.30151 ± 0.06041} & {\bfseries 0.29660 ± 0.06204} & {\bfseries 0.56457 ± 0.03550} \\
\hline


In [29]:
res

{'grey-knn-reg-asa-17:32-03-04': {'params': {'algorithm': 'ball_tree',
   'leaf_size': 27,
   'n_neighbors': 14,
   'p': 3.688252905206489,
   'weights': 'distance'},
  'scores': {'mae': (60.61168565239588, 1.752675893786236),
   'r2': (0.08455926941741058, 0.12997394614220695),
   'precision': (0.20821972892079016, 0.09487016186396942),
   'recall': (0.201052016364699, 0.04743372472323213),
   'f1_score': (0.16883359313871366, 0.04179276400763275),
   'roc_auc': (0.5175577122741698, 0.02988503511271758)},
  'predictions': [([292.5,
     225.5,
     297.0,
     278.5,
     234.5,
     187.5,
     186.5,
     255.5,
     220.5,
     255.0,
     137.5,
     220.0,
     116.5,
     262.5,
     266.5,
     93.0,
     245.5,
     218.0,
     188.5,
     60.5,
     246.0,
     279.5,
     251.0,
     206.5,
     216.5,
     243.5,
     224.0,
     101.0,
     184.0,
     38.0,
     215.0,
     66.0,
     215.0,
     209.0,
     174.5,
     39.5,
     154.5,
     166.5,
     227.5,
     120.5

In [77]:
def process_pred(pred, target="type"):
    y_true, y_pred = [np.concatenate(l) for l in zip(*pred)]

    if target != "type":
        y_true = reg_to_clf_target(y_true, target, 6)
        y_pred = reg_to_clf_target(y_pred, target, 6)
    return y_true, y_pred

In [78]:
from src.common.metrics import multiclass_roc_auc

with open("../results/exp-10.03/sev-clf.pkl", "rb") as file:
    res = pickle.load(file)

best_model = get_best_model(res, "clf")
pred = res[best_model]["predictions"]

y_true, y_pred = process_pred(pred, "type")
auc = multiclass_roc_auc(y_true, y_pred, 7)
type_labels = ["Acoustic-mnestic", "Dysarthria", "Sensory", "Efferent motor + Afferent motor", "Efferent motor", "Dynamic", "Afferent motor"]
sev_labels = ["Mild", "Mild-moderate", "Moderate", "Moderate-severe", "Severe", "Very severe"]
fig = make_cm(y_true, y_pred, type_labels, "clf", f"AUC = {auc:.4f}")

ValueError: Only one class present in y_true. ROC AUC score is not defined in that case.

In [79]:
np.unique(y_true, return_counts=True)

(array([0, 1, 2, 3, 4, 5]), array([ 78,  44,  49, 111,  50,  69]))

In [56]:
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import label_binarize

y_true = label_binarize(y_true, classes=list(range(6)))
y_pred = label_binarize(y_pred, classes=list(range(6)))
roc_auc_score(y_true, y_pred, average="weighted", multi_class="ovr")

ValueError: Only one class present in y_true. ROC AUC score is not defined in that case.

In [57]:
y_true

array([[0, 0, 0, 0, 0, 1],
       [0, 0, 1, 0, 0, 0],
       [0, 1, 0, 0, 0, 0],
       ...,
       [0, 1, 0, 0, 0, 0],
       [0, 0, 1, 0, 0, 0],
       [0, 1, 0, 0, 0, 0]])